In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('../data/arrecifes/clustering_evaluation.csv')

In [3]:
data.groupby('eps').silhouette_score.mean()

eps
5     0.971367
6     0.965069
7     0.940408
8     0.919704
9     0.868679
10    0.792129
Name: silhouette_score, dtype: float64